In [ ]:
import pandas as pd
import folium
import math

# ---- CSV laden ----
df = pd.read_csv("Baum999.csv", sep=";", encoding="utf-8")

# ---- EPSG:3857 -> WGS84 Umrechnung ----
R = 6378137.0
def mercator_to_wgs84(x, y):
    lon = (x / R) * (180 / math.pi)
    lat = (2 * math.atan(math.exp(y / R)) - math.pi/2) * (180 / math.pi)
    return lon, lat

df["lon"], df["lat"] = zip(*df.apply(lambda row: mercator_to_wgs84(row["x"], row["y"]), axis=1))

# ---- Parametrische Styles: Farben anhand "Kategorie" ----
styles = {
    "rot":   {"color": "red",   "fillColor": "red",   "radius": 4},
    "gruen": {"color": "green", "fillColor": "green", "radius": 4},
    "grün":  {"color": "green", "fillColor": "green", "radius": 4},
    "blau":  {"color": "blue",  "fillColor": "blue",  "radius": 4},
}

# ---- Karte erstellen (ohne Standardlayer, weil wir eigene hinzufügen) ----
m = folium.Map(location=[52.51, 13.4], zoom_start=11, control_scale=True, tiles=None)

# ---- Eigene Layer hinzufügen ----
# ---- Eigene Layer hinzufügen mit festen Namen ----
osm = folium.TileLayer(
    tiles="https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png",
    attr="© OpenStreetMap contributors",
    name="OpenStreetMap"
)
osm.get_name = lambda: "tilelayer_osm"
osm.add_to(m)

oepnv = folium.TileLayer(
    tiles="https://tileserver.memomaps.de/tilegen/{z}/{x}/{y}.png",
    attr="Map memomaps.de CC-BY-SA, map data © OpenStreetMap contributors",
    name="ÖPNVKarte"
)
oepnv.get_name = lambda: "tilelayer_oepnv"
oepnv.add_to(m)

cyclosm = folium.TileLayer(
    tiles="https://{s}.tile-cyclosm.openstreetmap.fr/cyclosm/{z}/{x}/{y}.png",
    attr="© CyclOSM / OSM contributors",
    name="CyclOSM"
)
cyclosm.get_name = lambda: "tilelayer_cyclosm"
cyclosm.add_to(m)

topplus = folium.TileLayer(
    tiles="http://sgx.geodatenzentrum.de/wmts_topplus_open/tile/1.0.0/web/default/WEBMERCATOR/{z}/{y}/{x}.png",
    attr="Map data: © http://www.govdata.de/dl-de/by-2-0 dl-de/by-2-0",
    name="TopPlusOpen Color"
)
topplus.get_name = lambda: "tilelayer_topplus"
topplus.add_to(m)


# ---- Marker mit Tooltip einfügen ----
for i, row in df.iterrows():
    kat = row["Kategorie"].strip().lower()
    s = styles.get(kat, {"color": "gray", "fillColor": "gray", "radius": 6})

    info = f"""
    <b>{row['Objektname']}</b><br>
    {row['Bezirk']}<br>
    {row['Art Botanisch']}
    """

    marker = folium.CircleMarker(
        location=[row["lat"], row["lon"]],
        radius=s["radius"],
        color=s["color"],
        fill=True,
        fill_color=s["fillColor"],
        fill_opacity=1.0,
        tooltip=info
    )
    
    # Feste ID vergeben, z. B. über Zeilenindex oder Spalte aus df
    marker.get_name = lambda n=f"circle_marker_{i}": n

    marker.add_to(m)


# Karte an alle Punkte anpassen
m.fit_bounds(m.get_bounds())

# Layer-Steuerung hinzufügen
folium.LayerControl().add_to(m)

# Fester Name um nicht im Versionskontrollsystem zu spammen
m.get_root().html.add_child(folium.Element('<div id="mapid" style="width:100%; height:100%;"></div>'))
m.get_name = lambda: "mapid"

# ---- Karte speichern ----
m.save("baumkarte.html")
print("Karte gespeichert als 'baumkarte.html'")
